<h2> connect to the cluster

In [1]:
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['p5-db-1', 'p5-db-2', 'p5-db-3'])
    session = cluster.connect()
except Exception as e:
    print(e)

('Unable to connect to any servers', {'172.18.0.2:9042': ConnectionRefusedError(111, "Tried connecting to [('172.18.0.2', 9042)]. Last error: Connection refused"), '172.18.0.3:9042': ConnectionRefusedError(111, "Tried connecting to [('172.18.0.3', 9042)]. Last error: Connection refused"), '172.18.0.4:9042': ConnectionRefusedError(111, "Tried connecting to [('172.18.0.4', 9042)]. Last error: Connection refused")})


<h2> create keyspace </h2>

In [14]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXIST weather
    WITH REPLICATION = 
    {'class' : 'SimpleStrategy','replication_factor' : 3}"""
    )
except Exception as e:
    print(e)

name 'session' is not defined
